# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [1]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [2]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

mkdir: data: File exists


In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31314    0 31314    0     0  67168      0 --:--:-- --:--:-- --:--:-- 67197


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70173    0 70173    0     0   635k      0 --:--:-- --:--:-- --:--:--  634k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

CHEAPER_MODEL = "gpt-4o-mini"
FULL_POWER_MODEL = "gpt-4o"

generator_llm = LangchainLLMWrapper(ChatOpenAI(model=CHEAPER_MODEL))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 59)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 59)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### 📝 Answer #1:

The three types of query synthesizers are:

1. SingleHopSpecificQuerySynthesizer: Generates straightforward, direct questions that can be answered from a single piece of information in the knowledge graph. For example, "What is the role of Stability AI?"

2. MultiHopAbstractQuerySynthesizer: Creates more complex questions that require connecting multiple pieces of information and providing a higher-level analysis or summary. These questions often ask about trends, patterns, or relationships between different concepts.

3. MultiHopSpecificQuerySynthesizer: Similar to the multi-hop abstract synthesizer, but generates questions that require specific details from multiple parts of the knowledge graph rather than abstract analysis. These questions might ask about specific connections between different events or developments.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [16]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What role has OpenAI played in the development...,[Vibes Based Development LLMs are really smart...,OpenAI was the only organization that had rele...,single_hop_specifc_query_synthesizer
1,What ethical concerns are associated with Stab...,[almost year old GPT-4 by now. OpenAI clearly ...,The ethical concerns surrounding Stable Diffus...,single_hop_specifc_query_synthesizer
2,Who is Simon Willison?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison is a blogger who discusses adva...,single_hop_specifc_query_synthesizer
3,What are the key insights about ChatGPT from t...,[is genuinely worth reading—especially the fir...,The author's blog in 2023 includes significant...,single_hop_specifc_query_synthesizer
4,Wht is Anthropic's Claude 3 series known for?,[Prompt driven app generation is a commodity a...,"Anthropic's Claude 3 series, launched in March...",single_hop_specifc_query_synthesizer
5,What role does careful training data design pl...,[<1-hop>\n\n“agents” as dependent on AGI itsel...,Careful training data design is crucial in the...,multi_hop_abstract_query_synthesizer
6,What advancements in OpenAI API access have co...,[<1-hop>\n\nyour camera feed with the model an...,"Advancements in OpenAI API access, particularl...",multi_hop_abstract_query_synthesizer
7,What were the LLM pricing trends in 2024 and h...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, LLM prices experienced a significant ...",multi_hop_abstract_query_synthesizer
8,What advancements did Anthropic make in their ...,[<1-hop>\n\nPrompt driven app generation is a ...,Anthropic made significant advancements in the...,multi_hop_specific_query_synthesizer
9,What impact did the launch of ChatGPT Pro have...,[<1-hop>\n\nyour camera feed with the model an...,The launch of ChatGPT Pro in 2023 had a signif...,multi_hop_specific_query_synthesizer


In [17]:
df = testset.to_pandas()

In [26]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(df.iloc[6].to_dict())

{   'reference': 'Advancements in OpenAI API access, particularly the '
                 'introduction of the new WebRTC API in December, have made it '
                 'significantly easier for developers to build web '
                 'applications that users can interact with via voice. This '
                 'ease of use has led to a surge in the creation of '
                 'interactive applications, which are now a commodity thanks '
                 'to the capabilities of models like GPT-4. The Chatbot Arena '
                 'team capitalized on this trend by introducing a leaderboard '
                 'that allows users to build the same interactive app twice '
                 'with different models and vote on the results. This not only '
                 'showcases the advancements in AI capabilities but also '
                 'enhances user interaction by providing a platform for '
                 'comparison and engagement with various models.',
    'reference_c

In [23]:
df.columns
df.head()
df.tail()
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_input          11 non-null     object
 1   reference_contexts  11 non-null     object
 2   reference           11 non-null     object
 3   synthesizer_name    11 non-null     object
dtypes: object(4)
memory usage: 484.0+ bytes


,user_input,reference_contexts,reference,synthesizer_name
count,11,11,11,11
unique,11,11,11,3
top,What role has OpenAI played in the development...,[Vibes Based Development LLMs are really smart...,OpenAI was the only organization that had rele...,single_hop_specifc_query_synthesizer
freq,1,1,1,5


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [27]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What advancements in large language models wer...,[Code may be the best application The ethics o...,"In 2024, significant advancements in large lan...",single_hop_specifc_query_synthesizer
1,Wht is the ChatGPT Code Interprter and how doe...,[Based Development As a computer scientist and...,The ChatGPT Code Interpreter is a pattern wher...,single_hop_specifc_query_synthesizer
2,What Simon Willison say about Weblog?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison’s Weblog discusses the breakthr...,single_hop_specifc_query_synthesizer
3,Who is Simon Willison in the context of AI adv...,[easy to follow. The rest of the document incl...,Simon Willison is a blogger who has written ex...,single_hop_specifc_query_synthesizer
4,How has increased competition influenced the p...,[<1-hop>\n\nPrompt driven app generation is a ...,Increased competition has significantly influe...,multi_hop_abstract_query_synthesizer
5,How have advancements in AI agents and real-ti...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI agents and real-time camera...,multi_hop_abstract_query_synthesizer
6,How are agents in AI and real-time camera feed...,[<1-hop>\n\nPrompt driven app generation is a ...,Recent AI advancements have seen the integrati...,multi_hop_abstract_query_synthesizer
7,How has the reduction in AI model pricing infl...,[<1-hop>\n\nPrompt driven app generation is a ...,The reduction in AI model pricing has signific...,multi_hop_abstract_query_synthesizer
8,What were the significant advancements in Larg...,[<1-hop>\n\nCode may be the best application T...,"In 2024, significant advancements in Large Lan...",multi_hop_specific_query_synthesizer
9,How does the concept of 'Vibes Based Developme...,[<1-hop>\n\nBased Development As a computer sc...,The concept of 'Vibes Based Development' highl...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [29]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years! v2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [30]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [31]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [34]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [35]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [36]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [38]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [39]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are an infuriatingly vague term that seems to be converging on "AI systems that can go away and act on your behalf." There are different interpretations, with some viewing agents as things that act on your behalf, resembling a travel agent model, while others think of them as LLMs given access to tools to solve problems in a loop. However, the term lacks a single, clear definition and there is skepticism about their utility due to challenges like gullibility.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [40]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [41]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

#### 📝 Activity Answer #2:

- `qa_evaluator`: This evaluator checks if the generated answer is correct and relevant to the question.
- `labeled_helpfulness_evaluator`: This evaluator checks if the generated answer is helpful to the user by considering the correct reference answer.
- `dope_or_nope_evaluator`: This evaluator checks if the generated answer is dope, lit, or cool.


## LangSmith Evaluation

In [42]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'kind-statement-55' at:
https://smith.langchain.com/o/a4d66ce1-dcf4-5067-a004-3babdd93af56/datasets/56966cf9-686a-4150-b0e7-3188881c5101/compare?selectedSessions=0fd0118e-9e16-4b2f-b284-901b34f9bafa




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,What are the implications of ChatGPT's develop...,The context suggests that the development of C...,None,The development of ChatGPT has significantly i...,1,1,1,2.628787,67e420be-233c-41df-9bd6-097252bdadd1,f604429b-b288-4e39-9547-7daf6e615588
1,"What advancements in AI technologies, particul...","In 2024, advancements in AI technologies, part...",None,"In 2024, advancements in AI technologies, part...",1,0,1,2.513873,44ab7b60-b4f2-4c43-8412-f3867396d15b,d546c711-17fa-4c24-8de8-4224be727fce
2,"How does the gullibility of LLMs, as discussed...",The gullibility of LLMs can be problematic bec...,None,"The gullibility of LLMs, including ChatGPT, si...",1,1,0,2.529611,f08ccd6f-100b-4eda-bfcc-a757e10d24bc,d362fbea-8a0c-4e78-b646-f0230f0b6be0
3,In what ways have the advancements in Large La...,I don't know.,None,Advancements in Large Language Models (LLMs) b...,0,0,0,1.255110,b03bed97-c89b-4dec-b898-711546bd9b0e,9f87a83f-f85e-4c4b-9f22-c359ada727d8
4,What role does the importance of training data...,I don't know.,None,The importance of training data is crucial in ...,0,0,0,0.847847,ceed9683-fe79-4b74-867f-adf8f1013f6f,db3937bc-e6e0-47a9-8ed6-d8c77d262b0f
5,What advancements in Large Language Models (LL...,"In 2023, significant advancements in Large Lan...",None,"In 2023, significant advancements in Large Lan...",0,0,1,2.078533,e71ad2e9-bbce-40aa-b445-be036f96d5ea,72ecd00e-05d6-48e8-b78f-3570bcfc9018
6,How do AI personal assistants enhance producti...,AI personal assistants can enhance productivit...,None,AI personal assistants enhance productivity by...,1,1,0,2.731768,4ed53636-2ef3-4782-b860-1c2ca46821c8,31ca3cc0-4f63-4505-b9c4-e42dcbfd0a39
7,In what ways can AI personal assistants enhanc...,AI personal assistants can enhance productivit...,None,AI personal assistants can enhance productivit...,1,0,0,3.051230,9ecc35ee-5d8a-4eb3-b6b3-73df8aaf0ddd,2a49f254-543f-4a71-bf27-2c7bc5315c0c
8,What role does OpenAI play in the advancements...,OpenAI is portrayed as a significant player in...,None,OpenAI is mentioned in the context of signific...,1,0,0,2.396373,d482d794-49d1-4459-8ca2-8c36bbb5a323,0fcd13bc-cf7b-4dd6-a0ba-b6bc96b76f3f
9,Wht is GPT-4?,GPT-4 is OpenAI's best language model that was...,None,GPT-4 is part of the advancements in Large Lan...,1,1,0,1.814253,395c8333-ed55-4ea5-97c8-12e9ee8aad68,4a385ef8-caee-44af-b40a-f532b048c955


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [43]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [44]:
rag_documents = docs

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### 📝 Answer #2:
Modifying chunk size affects performance in several key ways:
Larger chunks provide more context in each segment, which can help the model better understand the full context and relationships. However, larger chunks also mean fewer total chunks, which could reduce the granularity of retrieval. Chunk-based approaches work the best when a chunk represents a single, cohesive idea or concept. The chunk size needs to balance between having enough context (larger chunks) and precise retrieval (smaller chunks). Larger chunks may also increase computational overhead since more text needs to be processed at once (more context meaans higher latency for predictions and higher cost). The overlap between chunks helps maintain continuity and reduces the chance of breaking up important related information.


In [46]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### 📝 Answer #3:
If we use a larger embedding model, we will be able to capture more information about the text (and more nuanced information), which will help the model understand the context and relationships better. However, larger embedding models also mean higher computational cost. The choice of embedding model should strike a balance between performance and cost.

In [47]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [48]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [49]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [50]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Yo, agents are these AI systems that are supposed to act on your behalf, like a travel agent or a digital assistant. But here’s the kicker—they’re kinda vague and folks aren’t all on the same page about what they actually are. Some see them as LLMs with tools that can solve problems, while others think they’re just supposed to do stuff for you. The real issue? There’s this thing called gullibility. If the AI can’t tell what's real from what's fake, it’s gonna struggle making smart choices. So, yeah, agents are still in that “coming soon” phase, my friend!"

Finally, we can evaluate the new chain on the same test set!

In [52]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'advanced-dust-63' at:
https://smith.langchain.com/o/a4d66ce1-dcf4-5067-a004-3babdd93af56/datasets/56966cf9-686a-4150-b0e7-3188881c5101/compare?selectedSessions=bec42334-cd6d-47fb-985b-03d3f6c76ac7




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,What are the implications of ChatGPT's develop...,"Yo, the development of ChatGPT is like unlocki...",None,The development of ChatGPT has significantly i...,1,1,1,3.209898,67e420be-233c-41df-9bd6-097252bdadd1,99f08830-82f5-4503-aed0-7ed761a92f86
1,"What advancements in AI technologies, particul...","Yo, in 2024, AI tech took a major leap, especi...",None,"In 2024, advancements in AI technologies, part...",1,0,1,2.603651,44ab7b60-b4f2-4c43-8412-f3867396d15b,712fb366-4557-4be8-b6d7-7fd0a7bc186c
2,"How does the gullibility of LLMs, as discussed...","Yo, so here’s the scoop: the gullibility facto...",None,"The gullibility of LLMs, including ChatGPT, si...",1,1,1,3.307508,f08ccd6f-100b-4eda-bfcc-a757e10d24bc,285d9bc9-cfec-4c25-adee-9c060b0af4eb
3,In what ways have the advancements in Large La...,"Yo, here’s the lowdown! Companies like Meta ha...",None,Advancements in Large Language Models (LLMs) b...,1,1,1,3.698972,b03bed97-c89b-4dec-b898-711546bd9b0e,8d511f64-b612-4b6b-9169-d25c5f0ea91d
4,What role does the importance of training data...,"Yo, the importance of training data in the eth...",None,The importance of training data is crucial in ...,1,1,1,2.735730,ceed9683-fe79-4b74-867f-adf8f1013f6f,6f2ae590-e5b0-4ac1-8ef4-70772f38144d
5,What advancements in Large Language Models (LL...,"In 2023, we saw some seriously impressive adva...",None,"In 2023, significant advancements in Large Lan...",1,0,1,3.781190,e71ad2e9-bbce-40aa-b445-be036f96d5ea,d47f47f3-2bfe-41df-a65a-6ec9aca8f291
6,How do AI personal assistants enhance producti...,AI personal assistants can totally enhance pro...,None,AI personal assistants enhance productivity by...,1,1,1,3.352540,4ed53636-2ef3-4782-b860-1c2ca46821c8,0a506363-de4d-4d15-a9d7-efa9564b571c
7,In what ways can AI personal assistants enhanc...,AI personal assistants can boost productivity ...,None,AI personal assistants can enhance productivit...,1,0,1,2.941478,9ecc35ee-5d8a-4eb3-b6b3-73df8aaf0ddd,9315b49e-27ce-4a5c-aca0-faf8c001dfa4
8,What role does OpenAI play in the advancements...,OpenAI is a key player in the advancements of ...,None,OpenAI is mentioned in the context of signific...,0,0,1,2.446680,d482d794-49d1-4459-8ca2-8c36bbb5a323,0beb7c3b-39ae-4080-b280-4f1297009b87
9,Wht is GPT-4?,GPT-4 is a powerful language model released by...,None,GPT-4 is part of the advancements in Large Lan...,1,1,1,1.809937,395c8333-ed55-4ea5-97c8-12e9ee8aad68,9942e231-ef8b-4eb7-b745-48368437b8d5


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### 📝 Activity Answer #3:

Nope-application: https://loom.com/i/ce518771975d463e8fe34479ad31cc85
Dope-application: https://loom.com/i/143b1263def64c10bb0825f4969b1aee

The difference between the two chains is that the dope chain is more likely to generate a dope answer. This is because the dope chain has a prompt that encourages the model to generate a dope answer.

The fact that the dopeness metric changed isn't surprising because that was the whole point of the second chain version. So that's exactly what we'd expect to see. Wondersful!

The increase in correctness for the dope chain, on the other hand, was more surprising. Probably a weird artifact of the dataset. But if I were working in a production enviroment, I'd definitely want to dig further and see if I could figure out what led to that differential.



